In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn import datasets, neighbors, metrics, tree, svm, preprocessing, model_selection, ensemble


In [ ]:
%%time

# load labels
labels = pd.read_csv('../data/Lauren/Labels.csv')
#labels.head() # to display the first 5 lines of loaded data

# load data
df = pd.read_csv('../data/Lauren/500_PBMC_3p_LT_Chromium_X_metrics_summary.csv') # takes about 5min


CPU times: user 4min 17s, sys: 6.87 s, total: 4min 24s
Wall time: 4min 30s


In [ ]:
%%time
# filter out certain data
def gereral_data_filter(df, labels, filter_on, amount_higher_than):
    ## Filter info
    classes_count = labels.groupby(filter_on).count()
    classes_to_keep = list(classes_count[classes_count[classes_count.columns[-1]] >= amount_higher_than].index)
    keep_indices = labels[filter_on].isin(classes_to_keep)
    
    ## delete entries part of class that's too small
    return (df[keep_indices], labels[keep_indices])

df, labels = gereral_data_filter(df, labels, "cluster", 10)

CPU times: user 923 ms, sys: 3.89 s, total: 4.82 s
Wall time: 4.87 s


In [ ]:
# remove names of cells
df = df.drop(columns=["Unnamed: 0"])

In [ ]:
def train_linear_nn(data, class_column_name): # todo: give linear classifier as argument
    
    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train, y_train)
    return lin_clf